<h1 align="center">---- Data Science ----</h1>

----

**0) Packages**

In [2]:
using Pkg
Pkg.add("Clustering")
Pkg.add("VegaLite")
Pkg.add("VegaDatasets")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("JSON")
Pkg.add("CSV")
Pkg.add("Distances")

    Updating registry at `C:\Users\gabri\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  [aaaa29a8] + Clustering v0.14.3
  No Changes to `C:\Users\gabri\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
   Installed URIParser ── v0.4.1
   Installed JSONSchema ─ v1.0.1
   Installed NodeJS ───── v1.3.0
   Installed VegaLite ─── v2.6.0
   Installed Vega ─────── v2.3.1
    Updating `C:\Users\gabri\.julia\environments\v1.8\Project.toml`
  [112f6efa] + VegaLite v2.6.0
    Updating `C:\Users\gabri\.julia\environments\v1.8\Manifest.toml`
  [7d188eb4] + JSONSchema v1.0.1
  [2bd173c7] + NodeJS v1.3.0
  [30578b45] + URIParser v0.4.1
  [239c3e63] + Vega v2.3.1
  [112f6efa] + VegaLite v2.6.0
Precompiling project...
  ✓ NodeJS
  ✓ URIParser
  ✓ JSONSchema
  ✓ Vega
  ✓ VegaLite
  5 dependencies successfully precompiled in 10 seconds. 353 already precompiled. 5 skipped during auto due to previ

In [9]:
using Clustering
using VegaLite
using VegaDatasets
using Statistics
using JSON
using CSV
using Distances
using DataFrames

[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]


----

**1) Downloading and Inspecting Dataset**

In [32]:
# Download and Reading Dataset
download("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv", "newhouses.csv")
df = CSV.File("newhouses.csv") |> DataFrame

Row,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String15
1,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
2,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
3,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
4,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
5,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
6,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
7,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
8,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.12,241400.0,NEAR BAY
9,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY


In [33]:
# Listing Size and Features
println("Shape: $(size(df))")
println("Features: $(names(df))")

Shape: (20640, 10)
Features: ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "median_house_value", "ocean_proximity"]


In [34]:
# Stats Overview
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,longitude,-119.57,-124.35,-118.49,-114.31,0,Float64
2,latitude,35.6319,32.54,34.26,41.95,0,Float64
3,housing_median_age,28.6395,1.0,29.0,52.0,0,Float64
4,total_rooms,2635.76,2.0,2127.0,39320.0,0,Float64
5,total_bedrooms,537.871,1.0,435.0,6445.0,207,"Union{Missing, Float64}"
6,population,1425.48,3.0,1166.0,35682.0,0,Float64
7,households,499.54,1.0,409.0,6082.0,0,Float64
8,median_income,3.87067,0.4999,3.5348,15.0001,0,Float64
9,median_house_value,2.06856e5,14999.0,179700.0,500001.0,0,Float64


In [35]:
# Filling all missing values from "total_bedroom" feature with zeros
replace!(df.total_bedrooms, missing => 0)
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,longitude,-119.57,-124.35,-118.49,-114.31,0,Float64
2,latitude,35.6319,32.54,34.26,41.95,0,Float64
3,housing_median_age,28.6395,1.0,29.0,52.0,0,Float64
4,total_rooms,2635.76,2.0,2127.0,39320.0,0,Float64
5,total_bedrooms,532.476,0.0,431.0,6445.0,0,"Union{Missing, Float64}"
6,population,1425.48,3.0,1166.0,35682.0,0,Float64
7,households,499.54,1.0,409.0,6082.0,0,Float64
8,median_income,3.87067,0.4999,3.5348,15.0001,0,Float64
9,median_house_value,2.06856e5,14999.0,179700.0,500001.0,0,Float64


----

**2) K-Means Clustering**

In [36]:
?insertcols

search: insertcols insertcols!



```
insertcols(df::AbstractDataFrame[, col], (name=>val)::Pair...;
           after::Bool=false, makeunique::Bool=false, copycols::Bool=true)
```

Insert a column into a copy of `df` data frame using the [`insertcols!`](@ref) function and return the newly created data frame.

If `col` is omitted it is set to `ncol(df)+1` (the column is inserted as the last column).

# Arguments

  * `df` : the data frame to which we want to add columns
  * `col` : a position at which we want to insert a column, passed as an integer or a column name (a string or a `Symbol`); the column selected with `col` and columns following it are shifted to the right in `df` after the operation
  * `name` : the name of the new column
  * `val` : an `AbstractVector` giving the contents of the new column or a value of any type other than `AbstractArray` which will be repeated to fill a new vector; As a particular rule a values stored in a `Ref` or a `0`-dimensional `AbstractArray` are unwrapped and treated in the same way
  * `after` : if `true` columns are inserted after `col`
  * `makeunique` : defines what to do if `name` already exists in `df`; if it is `false` an error will be thrown; if it is `true` a new unique name will be generated by adding a suffix
  * `copycols` : whether vectors passed as columns should be copied

If `val` is an `AbstractRange` then the result of `collect(val)` is inserted.

If `df` is a `SubDataFrame` then it must have been created with `:` as column selector (otherwise an error is thrown). In this case the `copycols` keyword argument is ignored (i.e. the added column is always copied) and the parent data frame's column is filled with `missing` in rows that are filtered out by `df`.

If `df` isa `DataFrame` that has no columns and only values other than `AbstractVector` are passed then it is used to create a one-element column. If `df` isa `DataFrame` that has no columns and at least one `AbstractVector` is passed then its length is used to determine the number of elements in all created columns. In all other cases the number of rows in all created columns must match `nrow(df)`.

Metadata: this function preserves table-level and column-level `:note`-style metadata.

See also [`insertcols!`](@ref).

# Examples

```jldoctest
julia> df = DataFrame(a=1:3)
3×1 DataFrame
 Row │ a
     │ Int64
─────┼───────
   1 │     1
   2 │     2
   3 │     3

julia> insertcols(df, 1, :b => 'a':'c')
3×2 DataFrame
 Row │ b     a
     │ Char  Int64
─────┼─────────────
   1 │ a         1
   2 │ b         2
   3 │ c         3

julia> insertcols(df, :c => 2:4, :c => 3:5, makeunique=true)
3×3 DataFrame
 Row │ a      c      c_1
     │ Int64  Int64  Int64
─────┼─────────────────────
   1 │     1      2      3
   2 │     2      3      4
   3 │     3      4      5

julia> insertcols(df, :a, :d => 7:9, after=true)
3×2 DataFrame
 Row │ a      d
     │ Int64  Int64
─────┼──────────────
   1 │     1      7
   2 │     2      8
   3 │     3      9
```


In [37]:
# Preparing the Features
X = df[!, [:latitude, :longitude]]
clusters  = kmeans(Matrix(X)',  10)

cluster_df  = insertcols(
    df    # dataset  to receive the column
    , 1   # index of the new column
    , :clusterResult => clusters.assignments # values
)

Row,clusterResult,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
,Int64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String15
1,9,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
2,9,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
3,9,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
4,9,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
5,9,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
6,9,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
7,9,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
8,9,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.12,241400.0,NEAR BAY
9,9,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY


----

**3) Medoids Clustering**

In [41]:
X_matrix = Matrix(X)'
distance_matrix =  pairwise(
    Euclidean()  # algorithm (Euclidean Distances)
    , X_matrix   # df
    , X_matrix   # df
    , dims=2     # dimensions (pair => 2)
)

medoids_clusters = kmeans(distance_matrix, 10);

In [42]:
insertcols!(
    cluster_df
    ,  2
    , :medoidsClusterResult => medoids_clusters.assignments
)

Row,clusterResult,medoidsClusterResult,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
,Int64,Int64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String15
1,9,2,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
2,9,2,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
3,9,2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
4,9,2,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
5,9,2,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
6,9,2,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
7,9,2,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
8,9,2,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.12,241400.0,NEAR BAY
9,9,2,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY


----

**4) Hierarchial Clustering**

In [43]:
hierarchial_cluster = hclust(distance_matrix)
hierarchial_cluster_cutree = cutree(hierarchial_cluster, k=10)

insertcols!(
    cluster_df
    , 3
    , :hierarchialCluster => hierarchial_cluster_cutree
)

Row,clusterResult,medoidsClusterResult,hierarchialCluster,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String15
1,9,2,1,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
2,9,2,1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
3,9,2,1,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
4,9,2,1,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
5,9,2,1,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
6,9,2,1,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
7,9,2,1,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
8,9,2,1,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.12,241400.0,NEAR BAY
9,9,2,1,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY


----

**5)  DBScan (the best method between the four ones)**

In [44]:
?dbscan

search: dbscan DbscanResult DbscanCluster



```
dbscan(D::AbstractMatrix, eps::Real, minpts::Int) -> DbscanResult
```

Perform DBSCAN algorithm using the distance matrix `D`.

# Arguments

The following options control which points would be considered *density reachable*:

  * `eps::Real`: the radius of a point neighborhood
  * `minpts::Int`: the minimum number of neighboring points (including itself)  to qualify a point as a density point.

---

```
dbscan(points::AbstractMatrix, radius::Real,
       [leafsize], [min_neighbors], [min_cluster_size]) -> Vector{DbscanCluster}
```

Cluster `points` using the DBSCAN (density-based spatial clustering of applications with noise) algorithm.

# Arguments

  * `points`: the $d×n$ matrix of points. `points[:, j]` is a $d$-dimensional coordinates of $j$-th point
  * `radius::Real`: query radius

Optional keyword arguments to control the algorithm:

  * `leafsize::Int` (defaults to 20): the number of points binned in each leaf node in the `KDTree`
  * `min_neighbors::Int` (defaults to 1): the minimum number of a *core* point neighbors
  * `min_cluster_size::Int` (defaults to 1): the minimum number of points in a valid cluster

# Example

```julia
points = randn(3, 10000)
# DBSCAN clustering, clusters with less than 20 points will be discarded:
clusters = dbscan(points, 0.05, min_neighbors = 3, min_cluster_size = 20)
```


In [45]:
dbscan_clara  = pairwise(
    SqEuclidean()      # algorithm
    , distance_matrix  # distance matrix
    , dims=2           # number  of dimensions
)

dbscans_result = dbscan(dbscan_clara, 0.05,  10);

DbscanResult([17, 15618, 15622, 15664, 15691, 15714, 15747, 15821, 15829, 15876, 15983], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [10, 11, 15, 10, 10, 11, 10, 10, 10, 10, 10])

In [47]:
insertcols!(
    cluster_df
    , 4
    , :dbscanCluster => dbscans_result.assignments
)

Row,clusterResult,medoidsClusterResult,hierarchialCluster,dbscanCluster,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String15
1,9,2,1,0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
2,9,2,1,0,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
3,9,2,1,0,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
4,9,2,1,0,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
5,9,2,1,0,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
6,9,2,1,0,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
7,9,2,1,0,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
8,9,2,1,0,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.12,241400.0,NEAR BAY
9,9,2,1,0,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
